###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de https://microdatos.dane.gov.co/index.php/catalog/643/get_microdata y corresponden a datos cuya fuente primaria es el Departamento Administrativo Nacional de Estadisticas - DANE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import zipfile
import os

### Descarga de set de datos y carga como df

Se descarga 33 conjuntos de datos, uno por cada departamento más Bogotá, que contiene los microdatos de los personas del territorio nacional del  censo de población y vivienda realizado en 2018.

In [3]:
#  Función para listar archivos de un zip anidado
def extract_person_files_from_zip(zip_path, department_files):

    '''
    Esta función recibe el archivo comprimido principal y una carpeta
    dentro de ese archivo. Luego, descomprime los archivos que contienen 'PER_A2' en su nombre.
    '''
    person_files = []

    # Abrir el archivo ZIP principal
    with zipfile.ZipFile(zip_path, 'r') as main_zip:
        for dept_zip, csv_files in department_files.items():
            # Verificar si el archivo del departamento está dentro del zip principal
            if dept_zip in main_zip.namelist():
                # Extraer el archivo ZIP del departamento
                with main_zip.open(dept_zip) as dept_zip_file:
                    with zipfile.ZipFile(dept_zip_file) as dept_zip_contents:
                        # Iterar sobre los archivos CSV del departamento
                        for csv_file in csv_files:
                            if 'PER_A2' in csv_file:
                                # Extraer y leer solo los archivos que contienen 'PER_A2'
                                with dept_zip_contents.open(csv_file) as csv_data:
                                    df = pd.read_csv(csv_data)
                                    person_files.append(df)
    return person_files


In [4]:
# Función para consolidar los DataFrames
"""Después de extraer los archivos CSV, esta función concatenará los DataFrames de personas en un único DataFrame."""

def consolidate_person_dataframes(person_files):
    # Concatenar todos los DataFrames en uno solo
    consolidated_df = pd.concat(person_files, ignore_index=True)
    return consolidated_df

In [5]:
# Funcion que ejecuta el flujo
def build_person_dataframe(zip_path, department_files):
    person_files = extract_person_files_from_zip(zip_path, department_files)
    consolidated_df = consolidate_person_dataframes(person_files)
    return consolidated_df

In [6]:
# Lista de departamentos y archivos
deparment_files = {
    '05_Antioquia_CSV.zip' : ['CNPV2018_1VIV_A2_05.CSV', 'CNPV2018_2HOG_A2_05.CSV', 'CNPV2018_3FALL_A2_05.CSV', 'CNPV2018_5PER_A2_05.CSV', 'CNPV2018_MGN_A2_05.CSV'],
    '08_Atlantico_CSV.zip' : ['CNPV2018_1VIV_A2_08.CSV', 'CNPV2018_2HOG_A2_08.CSV', 'CNPV2018_3FALL_A2_08.CSV', 'CNPV2018_5PER_A2_08.CSV', 'CNPV2018_MGN_A2_08.CSV'],
    '11_Bogota_CSV.zip' : ['CNPV2018_1VIV_A2_11.CSV', 'CNPV2018_2HOG_A2_11.CSV', 'CNPV2018_3FALL_A2_11.CSV', 'CNPV2018_5PER_A2_11.CSV', 'CNPV2018_MGN_A2_11.CSV'],
    '13_Bolivar_CSV.zip' : ['CNPV2018_1VIV_A2_13.CSV', 'CNPV2018_2HOG_A2_13.CSV', 'CNPV2018_3FALL_A2_13.CSV', 'CNPV2018_5PER_A2_13.CSV', 'CNPV2018_MGN_A2_13.CSV'],
    '15_Boyaca_CSV.zip' : ['CNPV2018_1VIV_A2_15.CSV', 'CNPV2018_2HOG_A2_15.CSV', 'CNPV2018_3FALL_A2_15.CSV', 'CNPV2018_5PER_A2_15.CSV', 'CNPV2018_MGN_A2_15.CSV'],
    '17_Caldas_CSV.zip' : ['CNPV2018_1VIV_A2_17.CSV', 'CNPV2018_2HOG_A2_17.CSV', 'CNPV2018_3FALL_A2_17.CSV', 'CNPV2018_5PER_A2_17.CSV', 'CNPV2018_MGN_A2_17.CSV'],
    '18_Caqueta_CSV.zip' : ['CNPV2018_1VIV_A2_18.CSV', 'CNPV2018_2HOG_A2_18.CSV', 'CNPV2018_3FALL_A2_18.CSV', 'CNPV2018_5PER_A2_18.CSV', 'CNPV2018_MGN_A2_18.CSV'],
    '19_Cauca_CSV.zip' : ['CNPV2018_1VIV_A2_19.CSV', 'CNPV2018_2HOG_A2_19.CSV', 'CNPV2018_3FALL_A2_19.CSV', 'CNPV2018_5PER_A2_19.CSV', 'CNPV2018_MGN_A2_19.CSV'],
    '20_Cesar_CSV.zip' : ['CNPV2018_1VIV_A2_20.CSV', 'CNPV2018_2HOG_A2_20.CSV', 'CNPV2018_3FALL_A2_20.CSV', 'CNPV2018_5PER_A2_20.CSV', 'CNPV2018_MGN_A2_20.CSV'],
    '23_Cordoba_CSV.zip' : ['CNPV2018_1VIV_A2_23.CSV', 'CNPV2018_2HOG_A2_23.CSV', 'CNPV2018_3FALL_A2_23.CSV', 'CNPV2018_5PER_A2_23.CSV', 'CNPV2018_MGN_A2_23.CSV'],
    '25_Cundinamarca_CSV.zip' : ['CNPV2018_1VIV_A2_25.CSV', 'CNPV2018_2HOG_A2_25.CSV', 'CNPV2018_3FALL_A2_25.CSV', 'CNPV2018_5PER_A2_25.CSV', 'CNPV2018_MGN_A2_25.CSV'],
    '27_Choco_CSV.zip' : ['CNPV2018_1VIV_A2_27.CSV', 'CNPV2018_2HOG_A2_27.CSV', 'CNPV2018_3FALL_A2_27.CSV', 'CNPV2018_5PER_A2_27.CSV', 'CNPV2018_MGN_A2_27.CSV'],
    '41_Huila_CSV.zip' : ['CNPV2018_1VIV_A2_41.CSV', 'CNPV2018_2HOG_A2_41.CSV', 'CNPV2018_3FALL_A2_41.CSV', 'CNPV2018_5PER_A2_41.CSV', 'CNPV2018_MGN_A2_41.CSV'],
    '44_La Guajira_CSV.zip' : ['CNPV2018_1VIV_A2_44.CSV', 'CNPV2018_2HOG_A2_44.CSV', 'CNPV2018_3FALL_A2_44.CSV', 'CNPV2018_5PER_A2_44.CSV', 'CNPV2018_MGN_A2_44.CSV'],
    '47_Magdalena_CSV.zip' : ['CNPV2018_1VIV_A2_47.CSV', 'CNPV2018_2HOG_A2_47.CSV', 'CNPV2018_3FALL_A2_47.CSV', 'CNPV2018_5PER_A2_47.CSV', 'CNPV2018_MGN_A2_47.CSV'],
    '50_Meta_CSV.zip' : ['CNPV2018_1VIV_A2_50.CSV', 'CNPV2018_2HOG_A2_50.CSV', 'CNPV2018_3FALL_A2_50.CSV', 'CNPV2018_5PER_A2_50.CSV', 'CNPV2018_MGN_A2_50.CSV'],
    '52_Narino_CSV.zip' : ['CNPV2018_1VIV_A2_52.CSV', 'CNPV2018_2HOG_A2_52.CSV', 'CNPV2018_3FALL_A2_52.CSV', 'CNPV2018_5PER_A2_52.CSV', 'CNPV2018_MGN_A2_52.CSV'],
    '54_NorteDeSantander_CSV.zip' : ['CNPV2018_1VIV_A2_54.CSV', 'CNPV2018_2HOG_A2_54.CSV', 'CNPV2018_3FALL_A2_54.CSV', 'CNPV2018_5PER_A2_54.CSV', 'CNPV2018_MGN_A2_54.CSV'],
    '63_Quindio_CSV.zip' : ['CNPV2018_1VIV_A2_63.CSV', 'CNPV2018_2HOG_A2_63.CSV', 'CNPV2018_3FALL_A2_63.CSV', 'CNPV2018_5PER_A2_63.CSV', 'CNPV2018_MGN_A2_63.CSV'],
    '66_Risaralda_CSV.zip' : ['CNPV2018_1VIV_A2_66.CSV', 'CNPV2018_2HOG_A2_66.CSV', 'CNPV2018_3FALL_A2_66.CSV', 'CNPV2018_5PER_A2_66.CSV', 'CNPV2018_MGN_A2_66.CSV'],
    '68_Santander_CSV.zip' : ['CNPV2018_1VIV_A2_68.CSV', 'CNPV2018_2HOG_A2_68.CSV', 'CNPV2018_3FALL_A2_68.CSV', 'CNPV2018_5PER_A2_68.CSV', 'CNPV2018_MGN_A2_68.CSV'],
    '70_Sucre_CSV.zip' : ['CNPV2018_1VIV_A2_70.CSV', 'CNPV2018_2HOG_A2_70.CSV', 'CNPV2018_3FALL_A2_70.CSV', 'CNPV2018_5PER_A2_70.CSV', 'CNPV2018_MGN_A2_70.CSV'],
    '73_Tolima_CSV.zip' : ['CNPV2018_1VIV_A2_73.CSV', 'CNPV2018_2HOG_A2_73.CSV', 'CNPV2018_3FALL_A2_73.CSV', 'CNPV2018_5PER_A2_73.CSV', 'CNPV2018_MGN_A2_73.CSV'],
    '76_ValleDelCauca_CSV.zip' : ['CNPV2018_1VIV_A2_76.CSV', 'CNPV2018_2HOG_A2_76.CSV', 'CNPV2018_3FALL_A2_76.CSV', 'CNPV2018_5PER_A2_76.CSV', 'CNPV2018_MGN_A2_76.CSV'],
    '81_Arauca_CSV.zip' : ['CNPV2018_1VIV_A2_81.CSV', 'CNPV2018_2HOG_A2_81.CSV', 'CNPV2018_3FALL_A2_81.CSV', 'CNPV2018_5PER_A2_81.CSV', 'CNPV2018_MGN_A2_81.CSV'],
    '85_Casanare_CSV.zip' : ['CNPV2018_1VIV_A2_85.CSV', 'CNPV2018_2HOG_A2_85.CSV', 'CNPV2018_3FALL_A2_85.CSV', 'CNPV2018_5PER_A2_85.CSV', 'CNPV2018_MGN_A2_85.CSV'],
    '86_Putumayo_CSV.zip' : ['CNPV2018_1VIV_A2_86.CSV', 'CNPV2018_2HOG_A2_86.CSV', 'CNPV2018_3FALL_A2_86.CSV', 'CNPV2018_5PER_A2_86.CSV', 'CNPV2018_MGN_A2_86.CSV'],
    '88_SanAndresProvidenciaYSantaCatalina_CSV.zip' : ['CNPV2018_1VIV_A2_88.CSV', 'CNPV2018_2HOG_A2_88.CSV', 'CNPV2018_3FALL_A2_88.CSV', 'CNPV2018_5PER_A2_88.CSV', 'CNPV2018_MGN_A2_88.CSV'],
    '91_Amazonas_CSV.zip' : ['CNPV2018_1VIV_A2_91.CSV', 'CNPV2018_2HOG_A2_91.CSV', 'CNPV2018_3FALL_A2_91.CSV', 'CNPV2018_5PER_A2_91.CSV', 'CNPV2018_MGN_A2_91.CSV'],
    '94_Guainia_CSV.zip' : ['CNPV2018_1VIV_A2_94.CSV', 'CNPV2018_2HOG_A2_94.CSV', 'CNPV2018_3FALL_A2_94.CSV', 'CNPV2018_5PER_A2_94.CSV', 'CNPV2018_MGN_A2_94.CSV'],
    '95_Guaviare_CSV.zip' : ['CNPV2018_1VIV_A2_95.CSV', 'CNPV2018_2HOG_A2_95.CSV', 'CNPV2018_3FALL_A2_95.CSV', 'CNPV2018_5PER_A2_95.CSV', 'CNPV2018_MGN_A2_95.CSV'],
    '97_Vaupes_CSV.zip' : ['CNPV2018_1VIV_A2_97.CSV', 'CNPV2018_2HOG_A2_97.CSV', 'CNPV2018_3FALL_A2_97.CSV', 'CNPV2018_5PER_A2_97.CSV', 'CNPV2018_MGN_A2_97.CSV'],
    '99_Vichada_CSV.zip' : ['CNPV2018_1VIV_A2_99.CSV', 'CNPV2018_2HOG_A2_99.CSV', 'CNPV2018_3FALL_A2_99.CSV', 'CNPV2018_5PER_A2_99.CSV', 'CNPV2018_MGN_A2_99.CSV']
}

In [7]:
# Ruta base donde están los ZIP
zip_path = '/content/drive/MyDrive/analytics_data_proyect/censo_2018.zip'

# Ejecutar flujo
censo_personas_2018_dane = build_person_dataframe(zip_path, dict(list(deparment_files.items())[4:16]) )

- Tamaño del dataset obtenido

In [8]:
censo_personas_2018_dane.shape

(12759202, 48)

- Estructura del dataset creado a partir de las descargas

In [9]:
censo_personas_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12759202 entries, 0 to 12759201
Data columns (total 48 columns):
 #   Column            Dtype  
---  ------            -----  
 0   TIPO_REG          int64  
 1   U_DPTO            int64  
 2   U_MPIO            int64  
 3   UA_CLASE          int64  
 4   COD_ENCUESTAS     int64  
 5   U_VIVIENDA        int64  
 6   P_NROHOG          float64
 7   P_NRO_PER         int64  
 8   P_SEXO            int64  
 9   P_EDADR           int64  
 10  P_PARENTESCOR     float64
 11  PA1_GRP_ETNIC     int64  
 12  PA11_COD_ETNIA    float64
 13  PA12_CLAN         float64
 14  PA21_COD_VITSA    float64
 15  PA22_COD_KUMPA    float64
 16  PA_HABLA_LENG     float64
 17  PA1_ENTIENDE      float64
 18  PB_OTRAS_LENG     float64
 19  PB1_QOTRAS_LENG   float64
 20  PA_LUG_NAC        int64  
 21  PA_VIVIA_5ANOS    float64
 22  PA_VIVIA_1ANO     float64
 23  P_ENFERMO         float64
 24  P_QUEHIZO_PPAL    float64
 25  PA_LO_ATENDIERON  float64
 26  PA1_CALIDAD_

- Analisis de las columnas para determinar su importancia para el proyecto

In [10]:
columns = censo_personas_2018_dane.columns
columns

Index(['TIPO_REG', 'U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS',
       'U_VIVIENDA', 'P_NROHOG', 'P_NRO_PER', 'P_SEXO', 'P_EDADR',
       'P_PARENTESCOR', 'PA1_GRP_ETNIC', 'PA11_COD_ETNIA', 'PA12_CLAN',
       'PA21_COD_VITSA', 'PA22_COD_KUMPA', 'PA_HABLA_LENG', 'PA1_ENTIENDE',
       'PB_OTRAS_LENG', 'PB1_QOTRAS_LENG', 'PA_LUG_NAC', 'PA_VIVIA_5ANOS',
       'PA_VIVIA_1ANO', 'P_ENFERMO', 'P_QUEHIZO_PPAL', 'PA_LO_ATENDIERON',
       'PA1_CALIDAD_SERV', 'CONDICION_FISICA', 'P_ALFABETA', 'PA_ASISTENCIA',
       'P_NIVEL_ANOSR', 'P_TRABAJO', 'P_EST_CIVIL', 'PA_HNV', 'PA1_THNV',
       'PA2_HNVH', 'PA3_HNVM', 'PA_HNVS', 'PA1_THSV', 'PA2_HSVH', 'PA3_HSVM',
       'PA_HFC', 'PA1_THFC', 'PA2_HFCH', 'PA3_HFCM', 'PA_UHNV', 'PA1_MES_UHNV',
       'PA2_ANO_UHNV'],
      dtype='object')

In [11]:
# Significado de columna según el diccionario de datos de los dataset
meaning_columns= {
    "TIPO_REG": "Tipo de registro", "U_DPTO": "Departamento","U_MPIO": "Municipio","UA_CLASE": "Clase","COD_ENCUESTAS": "Código Encuesta",
    "U_VIVIENDA": "Número de orden de la Vivienda", "P_NROHOG": "Número de orden del Hogar dentro de la vivienda", "P_NRO_PER": "Número de persona en el hogar", "P_SEXO": "Sexo", "P_EDADR": "Edad en Grupos Quinquenales",
    "P_PARENTESCOR": "Relación de parentesco con el jefe(a) del hogar (recodificada)", "PA1_GRP_ETNIC": "Reconocimiento étnico", "PA11_COD_ETNIA": "Pueblo indigena de pertenencia", "PA12_CLAN": "Clan de pertenencia", "PA21_COD_VITSA": "Vitsa de pertenencia", "PA22_COD_KUMPA": "Kumpania de pertenencia",
    "PA_HABLA_LENG": "Habla la lengua nativa de su pueblo", "PA1_ENTIENDE": "Entiende la lengua nativa de su pueblo", "PB_OTRAS_LENG": "Habla otra(s) lengua(s) nativa(s)", "PB1_QOTRAS_LENG": "Número de otra(s) lengua(s) nativa(s)",
    "PA_LUG_NAC": "Lugar de nacimiento", "PA_VIVIA_5ANOS": "Lugar de residencia hace 5 años", "PA_VIVIA_1ANO": "Lugar de residencia hace 12 meses", "P_ENFERMO": "Algun problema de salud en los últimos 30 días, sin hospitalización",
    "P_QUEHIZO_PPAL": "Tratamiento principal del problema de salud", "PA_LO_ATENDIERON": "Atención del problema de salud", "PA1_CALIDAD_SERV": "Calidad de la prestación del servicio de salud",     "CONDICION_FISICA": "Alguna dificultad en su vida diaria",
    "P_ALFABETA": "Sabe leer y escribir", "PA_ASISTENCIA": "Asistencia escolar (de forma presencial o virtual)",  "P_NIVEL_ANOSR": "Nivel educativo más alto alcanzado y último año o grado aprobado en ese nivel (recodificado)",
    "P_TRABAJO": "Que hizo durante la semana pasada", "P_EST_CIVIL": "Estado civil", "PA_HNV": "Ha tenido algún hijo(a) nacido vivo(a)","PA1_THNV": "Hijos(as) nacidos vivos: Total",
    "PA2_HNVH": "Hijos(as) nacidos vivos: Hombres","PA3_HNVM": "Hijos(as) nacidos vivos: Mujeres", "PA_HNVS": "Hijos(as) sobrevivientes", "PA1_THSV": "Hijos(as) sobrevivientes: Total",
    "PA2_HSVH": "Hijos(as) sobrevivientes: Hombres","PA3_HSVM": "Hijos(as) sobrevivientes: Mujeres", "PA_HFC": "Hijos(as) viven actualmente fuera de Colombia", "PA1_THFC": "Hijos(as) viven actualmente fuera de Colombia: Total",
    "PA2_HFCH": "Hijos(as) viven actualmente fuera de Colombia: Hombres", "PA3_HFCM": "Hijos(as) viven actualmente fuera de Colombia: Mujeres", "PA_UHNV": "Nacimiento último hijo(a) nacido(a) vivo(a)",
    "PA1_MES_UHNV": "Nacimiento último hijo(a) nacido(a) vivo(a): Mes",  "PA2_ANO_UHNV": "Nacimiento último hijo(a) nacido(a) vivo(a): Año"
}
for col in meaning_columns:
  print(f'{col}: {meaning_columns[col]}')

TIPO_REG: Tipo de registro
U_DPTO: Departamento
U_MPIO: Municipio
UA_CLASE: Clase
COD_ENCUESTAS: Código Encuesta
U_VIVIENDA: Número de orden de la Vivienda
P_NROHOG: Número de orden del Hogar dentro de la vivienda
P_NRO_PER: Número de persona en el hogar
P_SEXO: Sexo
P_EDADR: Edad en Grupos Quinquenales
P_PARENTESCOR: Relación de parentesco con el jefe(a) del hogar (recodificada)
PA1_GRP_ETNIC: Reconocimiento étnico
PA11_COD_ETNIA: Pueblo indigena de pertenencia
PA12_CLAN: Clan de pertenencia
PA21_COD_VITSA: Vitsa de pertenencia
PA22_COD_KUMPA: Kumpania de pertenencia
PA_HABLA_LENG: Habla la lengua nativa de su pueblo
PA1_ENTIENDE: Entiende la lengua nativa de su pueblo
PB_OTRAS_LENG: Habla otra(s) lengua(s) nativa(s)
PB1_QOTRAS_LENG: Número de otra(s) lengua(s) nativa(s)
PA_LUG_NAC: Lugar de nacimiento
PA_VIVIA_5ANOS: Lugar de residencia hace 5 años
PA_VIVIA_1ANO: Lugar de residencia hace 12 meses
P_ENFERMO: Algun problema de salud en los últimos 30 días, sin hospitalización
P_QUEHIZO

In [12]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {censo_personas_2018_dane[col].nunique()}')

TIPO_REG: 1
U_DPTO: 11
U_MPIO: 362
UA_CLASE: 3
COD_ENCUESTAS: 3800762
U_VIVIENDA: 1744
P_NROHOG: 20
P_NRO_PER: 3295
P_SEXO: 2
P_EDADR: 21
P_PARENTESCOR: 5
PA1_GRP_ETNIC: 7
PA11_COD_ETNIA: 119
PA12_CLAN: 24
PA21_COD_VITSA: 8
PA22_COD_KUMPA: 12
PA_HABLA_LENG: 3
PA1_ENTIENDE: 3
PB_OTRAS_LENG: 3
PB1_QOTRAS_LENG: 10
PA_LUG_NAC: 4
PA_VIVIA_5ANOS: 5
PA_VIVIA_1ANO: 5
P_ENFERMO: 3
P_QUEHIZO_PPAL: 10
PA_LO_ATENDIERON: 3
PA1_CALIDAD_SERV: 4
CONDICION_FISICA: 2
P_ALFABETA: 3
PA_ASISTENCIA: 3
P_NIVEL_ANOSR: 11
P_TRABAJO: 10
P_EST_CIVIL: 8
PA_HNV: 3
PA1_THNV: 25
PA2_HNVH: 25
PA3_HNVM: 24
PA_HNVS: 3
PA1_THSV: 25
PA2_HSVH: 19
PA3_HSVM: 18
PA_HFC: 3
PA1_THFC: 19
PA2_HFCH: 16
PA3_HFCM: 15
PA_UHNV: 2
PA1_MES_UHNV: 13
PA2_ANO_UHNV: 87


In [13]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in columns:
  l = censo_personas_2018_dane[col].nunique()
  if l < 50:
    print(f'{col}: {censo_personas_2018_dane[col].unique()}')

TIPO_REG: [5]
U_DPTO: [15 17 18 19 20 23 25 27 41 47 50]
UA_CLASE: [1 3 2]
P_NROHOG: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. nan 14. 15. 16. 17.
 18. 19. 20.]
P_SEXO: [2 1]
P_EDADR: [13 11  4  3  1  5 12  9 17 16 14 15  6  7  2  8 10 18 19 20 21]
P_PARENTESCOR: [ 1.  3.  4.  2.  5. nan]
PA1_GRP_ETNIC: [6 9 5 1 3 4 2]
PA12_CLAN: [nan 99. 15. 22. 21. 20. 23.  2. 19.  1.  4.  9.  3.  6. 13.  7. 18.  8.
 17. 12. 10. 16. 11. 14.  5.]
PA21_COD_VITSA: [nan  9.  2.  5.  1.  7.  4.  3.  6.]
PA22_COD_KUMPA: [nan  5.  1.  2. 11. 99.  7.  4.  6. 10.  8.  3.  9.]
PA_HABLA_LENG: [nan  2.  1.  9.]
PA1_ENTIENDE: [nan  2.  1.  9.]
PB_OTRAS_LENG: [nan  2.  1.  9.]
PB1_QOTRAS_LENG: [nan  2.  1.  5. 99.  3.  7.  4.  6.  9.  8.]
PA_LUG_NAC: [1 2 9 3]
PA_VIVIA_5ANOS: [ 2.  1.  3.  9.  4. nan]
PA_VIVIA_1ANO: [ 2.  1.  3.  9.  4. nan]
P_ENFERMO: [ 2.  1.  9. nan]
P_QUEHIZO_PPAL: [nan  1.  3.  7.  2.  9.  4.  8.  6.  5.  0.]
PA_LO_ATENDIERON: [nan  1.  2.  9.]
PA1_CALIDAD_SERV: [nan  3.  2.  4.  1

In [14]:
censo_personas_2018_dane.columns

Index(['TIPO_REG', 'U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS',
       'U_VIVIENDA', 'P_NROHOG', 'P_NRO_PER', 'P_SEXO', 'P_EDADR',
       'P_PARENTESCOR', 'PA1_GRP_ETNIC', 'PA11_COD_ETNIA', 'PA12_CLAN',
       'PA21_COD_VITSA', 'PA22_COD_KUMPA', 'PA_HABLA_LENG', 'PA1_ENTIENDE',
       'PB_OTRAS_LENG', 'PB1_QOTRAS_LENG', 'PA_LUG_NAC', 'PA_VIVIA_5ANOS',
       'PA_VIVIA_1ANO', 'P_ENFERMO', 'P_QUEHIZO_PPAL', 'PA_LO_ATENDIERON',
       'PA1_CALIDAD_SERV', 'CONDICION_FISICA', 'P_ALFABETA', 'PA_ASISTENCIA',
       'P_NIVEL_ANOSR', 'P_TRABAJO', 'P_EST_CIVIL', 'PA_HNV', 'PA1_THNV',
       'PA2_HNVH', 'PA3_HNVM', 'PA_HNVS', 'PA1_THSV', 'PA2_HSVH', 'PA3_HSVM',
       'PA_HFC', 'PA1_THFC', 'PA2_HFCH', 'PA3_HFCM', 'PA_UHNV', 'PA1_MES_UHNV',
       'PA2_ANO_UHNV'],
      dtype='object')

In [15]:
len(censo_personas_2018_dane.columns)

48

In [16]:
# Eliminar columnas que se considera innecesarias para el proyecto
no_important_col = ['COD_ENCUESTAS','U_VIVIENDA', 'P_NROHOG', 'PA12_CLAN', 'PA21_COD_VITSA', 'PA22_COD_KUMPA', 'PA_HABLA_LENG', 'PA1_ENTIENDE', 'PB_OTRAS_LENG', 'PB1_QOTRAS_LENG',
 'PA_VIVIA_5ANOS', 'PA_VIVIA_1ANO', 'P_ENFERMO', 'P_QUEHIZO_PPAL', 'PA_LO_ATENDIERON', 'PA1_CALIDAD_SERV', 'PA_HNV', 'PA2_HNVH', 'PA3_HNVM', 'PA_HNVS', 'PA2_HSVH',
'PA3_HSVM', 'PA_HFC', 'PA2_HFCH', 'PA3_HFCM', 'PA1_MES_UHNV', 'PA2_ANO_UHNV']

censo_personas_2018_dane = censo_personas_2018_dane.drop(no_important_col, axis=1)
censo_personas_2018_dane.head()

,TIPO_REG,U_DPTO,U_MPIO,UA_CLASE,P_NRO_PER,P_SEXO,P_EDADR,P_PARENTESCOR,PA1_GRP_ETNIC,PA11_COD_ETNIA,...,CONDICION_FISICA,P_ALFABETA,PA_ASISTENCIA,P_NIVEL_ANOSR,P_TRABAJO,P_EST_CIVIL,PA1_THNV,PA1_THSV,PA1_THFC,PA_UHNV
0,5,15,1,1,1,2,13,1.0,6,NaN,...,2.0,1.0,2.0,2.0,1.0,5.0,10.0,10.0,NaN,9.0
1,5,15,1,1,1,2,11,1.0,6,NaN,...,2.0,1.0,2.0,2.0,1.0,4.0,3.0,3.0,NaN,9.0
2,5,15,1,1,2,2,4,3.0,6,NaN,...,2.0,1.0,2.0,4.0,7.0,7.0,1.0,1.0,NaN,1.0
3,5,15,1,1,3,1,3,4.0,6,NaN,...,2.0,1.0,1.0,3.0,6.0,7.0,NaN,NaN,NaN,NaN
4,5,15,1,1,4,2,1,4.0,6,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
censo_personas_2018_dane.shape

(12759202, 21)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios y los departamentos están almacenados por aparte y en formato int64


In [18]:
# valores codigo departamento
print(f'Valores unicos codigos depto: {censo_personas_2018_dane["U_DPTO"].unique()}')
print(f'Valores unicos codigos mpios: {censo_personas_2018_dane["U_MPIO"].unique()}')

Valores unicos codigos depto: [15 17 18 19 20 23 25 27 41 47 50]
Valores unicos codigos mpios: [  1  22  47  51  87  90  92  97 104 106 109 114 131 135 162 172 176 180
 183 185 187 189 204 212 215 218 223 224 226 232 236 238 244 248 272 276
 293 296 299 317 322 325 332 362 367 368 377 380 401 403 407 425 442 455
 464 466 469 476 480 491 494 500 507 511 514 516 518 522 531 533 537 542
 550 572 580 599 600 621 632 638 646 660 664 667 673 676 681 686 690 693
 696 720 723 740 753 755 757 759 761 762 763 764 774 776 778 790 798 804
 806 808 810 814 816 820 822 832 835 837 839 842 861 879 897  13  42  50
  88 174 388 433 444 446 486 495 513 524 541 614 616 653 662 665 777 867
 873 877  29  94 150 205 247 256 410 460 479 592 610 756 785 860  75 100
 110 130 137 142 290 300 318 355 364 392 397 418 450 473 517 532 548 573
 585 622 698 701 743 760 780 807 809 821 824 845  11  32  45  60 175 178
 228 250 295 310 383 400 443 570 710 750 770 787  68  79 168 182 350 417
 419 555 574 586 670 672 675 

- Longitud de los códigos de departamentos y municipios en el df

In [19]:
# Convertir a strings
censo_personas_2018_dane[['U_DPTO','U_MPIO']] = censo_personas_2018_dane[['U_DPTO','U_MPIO']].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_depto = censo_personas_2018_dane['U_DPTO'].apply(len)
longitud_mpio = censo_personas_2018_dane['U_MPIO'].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los dptos tiene longitudes de :{longitud_depto.unique()} caracteres')
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los dptos tiene longitudes de :[2] caracteres
Los códigos de los mpios tiene longitudes de :[1 2 3] caracteres


- Convertir los códigos de los departamentos en un string de 2 digitos y los de los municipios en un string de 3 digitos para que coincidan con el formato del código almacenado en la base de datos

Rellenamos con ceros a la izquierda aquellos códigos del departamento que solo tienen un dígito y aquellos códigos de los municipios que solo tienen 1 y 2 dígitos


In [20]:
# Función para formatear los códigos
def formatear_codigo(codigo, longitud):
    return codigo.zfill(longitud)

# Aplicar la función a las columnas y crear la nueva columna
censo_personas_2018_dane['U_DPTO'] = censo_personas_2018_dane['U_DPTO'].apply(formatear_codigo, args=(2,))
censo_personas_2018_dane['U_MPIO'] = censo_personas_2018_dane['U_MPIO'].apply(formatear_codigo, args=(3,))

- Crear nueva columna que contenga el codigo del departamento mas el codigo del municipio

In [21]:
# Nueva columna de código de municipio con longitud de 5 caracteres
censo_personas_2018_dane['COD_DEP_MPIO'] = censo_personas_2018_dane['U_DPTO'] +censo_personas_2018_dane['U_MPIO']

# Verificar
censo_personas_2018_dane['COD_DEP_MPIO']

,COD_DEP_MPIO
0,15001
1,15001
2,15001
3,15001
4,15001
...,...
12759197,50711
12759198,50711
12759199,50711
12759200,50711


In [22]:
# Eliminar columnas 'U_DPTO' y 'U_MPIO' que ya no son necesarias
censo_personas_2018_dane = censo_personas_2018_dane.drop(['U_DPTO', 'U_MPIO'], axis=1)
censo_personas_2018_dane.head(3)

,TIPO_REG,UA_CLASE,P_NRO_PER,P_SEXO,P_EDADR,P_PARENTESCOR,PA1_GRP_ETNIC,PA11_COD_ETNIA,PA_LUG_NAC,CONDICION_FISICA,P_ALFABETA,PA_ASISTENCIA,P_NIVEL_ANOSR,P_TRABAJO,P_EST_CIVIL,PA1_THNV,PA1_THSV,PA1_THFC,PA_UHNV,COD_DEP_MPIO
0,5,1,1,2,13,1.0,6,NaN,1,2.0,1.0,2.0,2.0,1.0,5.0,10.0,10.0,NaN,9.0,15001
1,5,1,1,2,11,1.0,6,NaN,1,2.0,1.0,2.0,2.0,1.0,4.0,3.0,3.0,NaN,9.0,15001
2,5,1,2,2,4,3.0,6,NaN,1,2.0,1.0,2.0,4.0,7.0,7.0,1.0,1.0,NaN,1.0,15001


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [23]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [24]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [25]:
# Comparar listas de códigos
compare_lists(censo_personas_2018_dane['COD_DEP_MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_by_type_of_crime_UnidadVictimas", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_by_type_of_crime_UnidadVictimas que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [26]:
# Adicionar columna para trazabilidad de la fuente
censo_personas_2018_dane['SOURCE_ID'] = censo_personas_2018_dane['COD_DEP_MPIO'].apply(
    lambda x: 58 if str(x).startswith('15') else
              59 if str(x).startswith('17') else
              60 if str(x).startswith('18') else
              61 if str(x).startswith('19') else
              62 if str(x).startswith('20') else
              63 if str(x).startswith('23') else
              64 if str(x).startswith('25') else
              65 if str(x).startswith('27') else
              66 if str(x).startswith('41') else
              67 if str(x).startswith('44') else
              68 if str(x).startswith('47') else
              69 if str(x).startswith('50') else None
)

In [27]:
censo_personas_2018_dane['SOURCE_ID'].unique()

array([58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69])

In [30]:
#Estructura final del dataset a integrar a la base de datos
censo_personas_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12759202 entries, 0 to 12759201
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   TIPO_REG          int64  
 1   UA_CLASE          int64  
 2   P_NRO_PER         int64  
 3   P_SEXO            int64  
 4   P_EDADR           int64  
 5   P_PARENTESCOR     float64
 6   PA1_GRP_ETNIC     int64  
 7   PA11_COD_ETNIA    float64
 8   PA_LUG_NAC        int64  
 9   CONDICION_FISICA  float64
 10  P_ALFABETA        float64
 11  PA_ASISTENCIA     float64
 12  P_NIVEL_ANOSR     float64
 13  P_TRABAJO         float64
 14  P_EST_CIVIL       float64
 15  PA1_THNV          float64
 16  PA1_THSV          float64
 17  PA1_THFC          float64
 18  PA_UHNV           float64
 19  COD_DEP_MPIO      object 
 20  SOURCE_ID         int64  
dtypes: float64(12), int64(8), object(1)
memory usage: 2.0+ GB


## Salvar en archivo csv en el drive

In [31]:
# Guardar en archivos CSV en el drive
censo_personas_2018_dane.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/censo_personas_2018_dane_02.csv', index=False)